In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Aac2025"

# Connect to database via CRUD Module
db = CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True, errors='ignore')

## Debug
# print(f"Number of records retrieved: {len(df.to_dict(orient='records'))}")
# print(f"Columns: {df.columns}")


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # 1. Logo and Unique Identifier (FIXME: Ensure 'Lyth Hammad' is correct)
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
             style={'height':'100px', 'float':'right'}), 
    html.Center(html.B(html.H2('CS-340 Dashboard by Lyth Hammad'))),
    
    html.Hr(),
    
    # 2. Interactive Filtering Options (Radio Items)
    html.Div(
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset', # Default value is 'Reset'
            inline=True,
            labelStyle={'margin-right': '20px'}
        )
    ),
    
    html.Hr(),
    
    # 3. Interactive Data Table (with enhancements)
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         
                         # User-friendly table features (from previous step)
                         editable=False,
                         filter_action="native",
                         sort_action="native",
                         sort_mode="multi",
                         column_selectable="single",
                         row_selectable="single",
                         page_action="native",
                         page_current=0,
                         page_size=10
                        ),
    html.Br(),
    html.Hr(),
    # 4. Charts/Map Layout
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])



#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    # Base query: All dogs must be of animal_type 'Dog'
    base_query = {"animal_type": "Dog"}
    
    # Define the queries based on the selected filter type
    if filter_type == 'Water Rescue':
        query = {
            "$and": [
                base_query,
                {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}},
                {"sex_upon_outcome": "Intact Female"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {
            "$and": [
                base_query,
                {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}
            ]
        }
    elif filter_type == 'Disaster or Individual Tracking':
        query = {
            "$and": [
                base_query,
                {"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}},
                {"sex_upon_outcome": "Intact Male"},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}
            ]
        }
    elif filter_type == 'Reset':
        query = {}  # An empty dictionary returns every document in the collection
    else:
        query = base_query

    # Execute the read operation with the selected query
    data_list = db.read(query)
    
    # Convert the list of dictionaries (data_list) to a Pandas DataFrame
    dff = pd.DataFrame.from_records(data_list)
    
    # Clean the DataFrame by dropping the '_id' column if it exists
    dff.drop(columns=['_id'], inplace=True, errors='ignore')

    # Return the data in the format required by the DataTable
    return dff.to_dict('records')


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # This chart must update based on the filtered data (viewData)
    
    if viewData is None or not viewData:
        # Return a placeholder if no data is present
        return [html.Div("No data to display for the chart.", style={'padding': '20px', 'textAlign': 'center'})]

    # Convert the list of dictionaries (viewData) back to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create the Pie Chart showing the distribution of 'breed'
    pie_chart = dcc.Graph(            
        figure = px.pie(
            dff, 
            names='breed', 
            title='Distribution of Preferred Breeds in Filtered Data',
            hole=.3 # Optional: Makes it a donut chart
        )    
    )
    return [pie_chart]




# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # 1. Check for empty data (This prevents the size 0 error)
    if viewData is None or not viewData:
        # Return a simple placeholder map if no data is present
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            html.Div("No location data for selected animals.", style={'position': 'relative', 'top': '50%', 'textAlign': 'center'})
        ])]

    dff = pd.DataFrame.from_dict(viewData)
    
    # 2. Check for empty selection (Prevents list index out of range if no row is clicked)
    if not index:
        row = 0
    else:  
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 are location_lat and location_long based on your column list
            # Column 4 is breed, Column 9 is name
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://projectemail-fictionspray-3000.codio.io/proxy/8050/
